In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Serialization formats

In [2]:
%timeit pd.read_table("data/Main_final_cleaned_316.tsv")

1 loop, best of 3: 3.83 s per loop


In [3]:
%timeit pd.read_hdf("data/Main_final_cleaned_316.hdf")

1 loop, best of 3: 425 ms per loop


## Load Data

In [4]:
df = pd.read_hdf("data/Main_final_cleaned_316.hdf")
df[:5]

,trip_id,lat,lon,imo,timestamp_position,sog,name,dist_orig,dist_dest,port_id_orig,port_id_dest,cum_dist_orig,cum_dist_dest,time_to_arrive,time_from_departure,time_port_to_port
0,12124281464180571,40.644173,-74.122902,1212428,2016-06-06 19:28:16,6.9,MAGOTHY,457569.057001,3016.951519,285,316,542072.385284,22961.778699,0.145833,11.202083,11.347917
1,12124281464180571,40.711784,-74.122818,1212428,2016-06-06 21:15:36,0.0,MAGOTHY,464362.593692,4540.163553,285,316,549586.456525,9177.323661,0.071528,11.276389,11.347917
2,12124281464180571,40.859440,-73.946632,1212428,2016-06-06 16:57:17,6.8,MAGOTHY,485558.703834,25914.821769,285,316,512531.912831,55064.576092,0.250694,11.097222,11.347917
3,12124281464180571,40.936378,-73.906212,1212428,2016-06-05 15:57:49,5.3,MAGOTHY,494737.796234,34899.458219,285,316,499450.236029,59167.090988,1.292361,10.055556,11.347917
4,12124281464180571,40.945038,-73.908249,1212428,2016-06-05 16:42:06,6.3,MAGOTHY,495523.918761,35620.619295,285,316,500420.026386,57642.342082,1.261111,10.086806,11.347917


## Ranking: Find ports with most incoming trips

In [5]:
df_tmp = df.groupby(['port_id_orig'])['trip_id'].size().reset_index()
df_tmp.sort_values(by=0, ascending=False).head(5)

,port_id_orig,0
2,5,142334
54,195,66454
93,332,59067
80,285,55214
47,154,47478


In [6]:
del df_tmp

## Ranking 2: Find terminals with most incoming trips, for each terminal in 316

In [7]:
df_trips = pd.read_hdf("data/HIST_Trips.hdf")
df2 = df.merge(df_trips, on='trip_id', how='inner')

df_tmp = df2.groupby(['berth_in_id_dest','berth_in_id_orig'])['trip_id'].size().reset_index()
df_tmp.sort_values(by=['berth_in_id_dest',0], ascending=[True,False], inplace=True)
df_tmp[:5]

,berth_in_id_dest,berth_in_id_orig,0
13,APM Terminal,Damietta Container Terminal,19990
37,APM Terminal,Pasir Panjang Terminal,15112
0,APM Terminal,APM Algeciras,14925
43,APM Terminal,Salalah Container Terminal,12894
51,APM Terminal,Stromkaje Eurogate,11393


### Rank top 3

In [8]:
df_tmp2 = df_tmp.groupby(['berth_in_id_dest'],sort=False).head(3)
df_tmp2[:5]

,berth_in_id_dest,berth_in_id_orig,0
13,APM Terminal,Damietta Container Terminal,19990
37,APM Terminal,Pasir Panjang Terminal,15112
0,APM Terminal,APM Algeciras,14925
60,Bayonne,Ducan Dock E - L,393
62,Bayonne,Terminal PorteOceane,173


### Rank top N percentile

In [9]:
df_tmp3 = df_tmp.copy(deep=True)
df_tmp3['percent_traffic'] = df_tmp.groupby(['berth_in_id_dest'],sort=False)[0].apply(lambda x: x/sum(x))
df_tmp3['percent_traffic_cum'] = df_tmp3.groupby('berth_in_id_dest')['percent_traffic'].cumsum()
df_tmp4 = df_tmp3[df_tmp3['percent_traffic_cum']<=0.2]
df_tmp4[:5]

,berth_in_id_dest,berth_in_id_orig,0,percent_traffic,percent_traffic_cum
13,APM Terminal,Damietta Container Terminal,19990,0.104147,0.104147
37,APM Terminal,Pasir Panjang Terminal,15112,0.078733,0.182880
64,Brooklyn,Boston Port,2627,0.160046,0.160046
154,Global Marine Terminal,Brani Terminal,17247,0.164950,0.164950
211,IMTT Bayonne,Boston Port,1806,0.135048,0.135048


In [10]:
del df2
del df_tmp
del df_tmp2
del df_tmp3
del df_tmp4

## Lookup: Port Names

In [11]:
df_ports = pd.read_table("data/ports.tsv")
df3 = df.merge(df_ports[['port_ID','port_name']], left_on='port_id_orig',right_on='port_ID', how='left')
df3 = df3.merge(df_ports[['port_ID','port_name']], left_on='port_id_dest',right_on='port_ID', how='left', suffixes=['_orig','_dest'])
df3[:5]

,trip_id,lat,lon,imo,timestamp_position,sog,name,dist_orig,dist_dest,port_id_orig,port_id_dest,cum_dist_orig,cum_dist_dest,time_to_arrive,time_from_departure,time_port_to_port,port_ID_orig,port_name_orig,port_ID_dest,port_name_dest
0,12124281464180571,40.644173,-74.122902,1212428,2016-06-06 19:28:16,6.9,MAGOTHY,457569.057001,3016.951519,285,316,542072.385284,22961.778699,0.145833,11.202083,11.347917,285,Norfolk,316,Newark
1,12124281464180571,40.644173,-74.122902,1212428,2016-06-06 19:28:16,6.9,MAGOTHY,457569.057001,3016.951519,285,316,542072.385284,22961.778699,0.145833,11.202083,11.347917,285,Norfolk,316,New York
2,12124281464180571,40.711784,-74.122818,1212428,2016-06-06 21:15:36,0.0,MAGOTHY,464362.593692,4540.163553,285,316,549586.456525,9177.323661,0.071528,11.276389,11.347917,285,Norfolk,316,Newark
3,12124281464180571,40.711784,-74.122818,1212428,2016-06-06 21:15:36,0.0,MAGOTHY,464362.593692,4540.163553,285,316,549586.456525,9177.323661,0.071528,11.276389,11.347917,285,Norfolk,316,New York
4,12124281464180571,40.859440,-73.946632,1212428,2016-06-06 16:57:17,6.8,MAGOTHY,485558.703834,25914.821769,285,316,512531.912831,55064.576092,0.250694,11.097222,11.347917,285,Norfolk,316,Newark


## Summarising data

In [12]:
df = pd.read_hdf("/Users/anil/src/knife/Geo_data/data/Points_In_Polygon.hdf")
df[np.all([df.imo==7905584, pd.notnull(df.Unique_ID)], axis=0)][:10]
df.sort_values(by=['imo','timestamp_position','Unique_ID'],inplace=True)
df = df.drop_duplicates(subset=['imo','timestamp_position'],keep='first')
df[:5]

,imo,name,Unique_ID,lat,lon,timestamp_position,sog
2372724,7905584,Shahab 14,NaN,27.074041,56.227280,2016-09-23 15:12:11,0.0
2682658,7905584,Shahab 14,NaN,27.074053,56.227268,2016-09-23 15:27:10,0.1
1145379,7905584,Shahab 14,NaN,27.074072,56.227219,2016-09-23 16:45:09,0.1
1680831,7905584,Shahab 14,NaN,27.074142,56.227215,2016-09-23 16:57:10,0.1
2051865,7905584,Shahab 14,NaN,27.074600,56.226955,2016-09-23 17:57:10,0.1


In [13]:
df_1 = df.shift(1)
df_1[:5]

,imo,name,Unique_ID,lat,lon,timestamp_position,sog
2372724,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2682658,7905584.0,Shahab 14,NaN,27.074041,56.227280,2016-09-23 15:12:11,0.0
1145379,7905584.0,Shahab 14,NaN,27.074053,56.227268,2016-09-23 15:27:10,0.1
1680831,7905584.0,Shahab 14,NaN,27.074072,56.227219,2016-09-23 16:45:09,0.1
2051865,7905584.0,Shahab 14,NaN,27.074142,56.227215,2016-09-23 16:57:10,0.1


In [14]:
df['flag'] = np.any([df.imo!=df_1.imo,df.Unique_ID!=df_1.Unique_ID], axis=0).astype(int)
df[pd.notnull(df['Unique_ID'])][:5]

,imo,name,Unique_ID,lat,lon,timestamp_position,sog,flag
3174221,7905584,Shahab 14,3792.0,27.108385,56.067524,2016-10-10 20:27:51,0.1,1
2990393,7905584,Shahab 14,3792.0,27.108431,56.067543,2016-10-10 20:35:41,0.1,0
590657,7905584,Shahab 14,3792.0,27.108404,56.067558,2016-10-10 20:59:02,0.1,0
2611421,7905584,Shahab 14,3792.0,27.108416,56.067516,2016-10-10 21:13:21,0.1,0
1435282,7905584,Shahab 14,3792.0,27.108349,56.067520,2016-10-10 21:29:21,0.1,0


In [15]:
df.flag = df.flag.cumsum()
df[pd.notnull(df['Unique_ID'])][:5]

,imo,name,Unique_ID,lat,lon,timestamp_position,sog,flag
3174221,7905584,Shahab 14,3792.0,27.108385,56.067524,2016-10-10 20:27:51,0.1,127
2990393,7905584,Shahab 14,3792.0,27.108431,56.067543,2016-10-10 20:35:41,0.1,127
590657,7905584,Shahab 14,3792.0,27.108404,56.067558,2016-10-10 20:59:02,0.1,127
2611421,7905584,Shahab 14,3792.0,27.108416,56.067516,2016-10-10 21:13:21,0.1,127
1435282,7905584,Shahab 14,3792.0,27.108349,56.067520,2016-10-10 21:29:21,0.1,127


In [16]:
df_head = df.groupby('flag').head(1)
df_tail = df.groupby('flag').tail(1)
df_head[:5]

,imo,name,Unique_ID,lat,lon,timestamp_position,sog,flag
2372724,7905584,Shahab 14,NaN,27.074041,56.227280,2016-09-23 15:12:11,0.0,1
2682658,7905584,Shahab 14,NaN,27.074053,56.227268,2016-09-23 15:27:10,0.1,2
1145379,7905584,Shahab 14,NaN,27.074072,56.227219,2016-09-23 16:45:09,0.1,3
1680831,7905584,Shahab 14,NaN,27.074142,56.227215,2016-09-23 16:57:10,0.1,4
2051865,7905584,Shahab 14,NaN,27.074600,56.226955,2016-09-23 17:57:10,0.1,5


In [17]:
df_tmp = pd.merge(df_head,df_tail,on=['imo','name','flag','Unique_ID'],suffixes=['_in','_out'])
df_voyages = df_tmp[pd.notnull(df_tmp.Unique_ID)]
df_voyages[:5]

,imo,name,Unique_ID,lat_in,lon_in,timestamp_position_in,sog_in,flag,lat_out,lon_out,timestamp_position_out,sog_out
126,7905584,Shahab 14,3792.0,27.108385,56.067524,2016-10-10 20:27:51,0.1,127,27.108412,56.067524,2016-10-11 09:41:22,0.1
902,7905584,Shahab 14,3792.0,27.107901,56.067810,2016-10-31 13:58:49,0.1,903,27.107988,56.067734,2016-11-01 02:28:46,0.1
1537,7905584,Shahab 14,3792.0,27.104183,56.069904,2016-11-28 15:28:00,0.0,1538,27.104074,56.069778,2016-11-29 06:29:52,0.8
7235,8209729,MSC Perle,61.0,22.737938,69.690994,2016-11-22 04:29:08,0.9,7236,22.738213,69.690681,2016-11-22 20:40:03,0.0
7237,8209729,MSC Perle,3761.0,25.023144,55.050079,2016-11-26 07:45:02,0.0,7238,25.022978,55.049988,2016-11-27 00:59:02,0.0


### Summarising data: slightly complex use-case

In [18]:
df_voyages[np.all([df_voyages.imo==8420907, df_voyages.timestamp_position_in>=datetime(2016,10,1), \
                   df_voyages.timestamp_position_in<=datetime(2016,10,3)], axis=0)][:5]

,imo,name,Unique_ID,lat_in,lon_in,timestamp_position_in,sog_in,flag,lat_out,lon_out,timestamp_position_out,sog_out
12132,8420907,MSC Jasmine,3801.0,16.942984,54.001366,2016-10-02 11:59:24,0.0,12133,16.943050,54.001366,2016-10-02 14:25:32,0.0
12134,8420907,MSC Jasmine,3801.0,16.942984,54.001484,2016-10-02 14:55:30,0.0,12135,16.943132,54.001366,2016-10-02 18:49:33,0.0
12136,8420907,MSC Jasmine,3801.0,16.943033,54.001415,2016-10-02 19:28:32,0.0,12137,16.942633,54.001717,2016-10-03 08:58:30,0.7


In [19]:
df_voyages_1 = df_voyages.shift(-1)
df_voyages_1['identical_to_prv'] = np.all([df_voyages.imo==df_voyages_1.imo, df_voyages.Unique_ID==df_voyages_1.Unique_ID, (df_voyages_1['timestamp_position_in'] - df_voyages['timestamp_position_out']).apply(lambda x: x.total_seconds()/60.0/60 <= 3)], axis=0)==True
df_voyages['identical_to_prv'] = df_voyages_1['identical_to_prv'].shift(1)
ix_identical_next = df_voyages_1[df_voyages_1['identical_to_prv']].index
df_voyages.ix[ix_identical_next,['timestamp_position_out']] = df_voyages_1[df_voyages_1['identical_to_prv']]['timestamp_position_out']
df_voyages = df_voyages[~(df_voyages['identical_to_prv']==True)]
df_voyages[:5]

,imo,name,Unique_ID,lat_in,lon_in,timestamp_position_in,sog_in,flag,lat_out,lon_out,timestamp_position_out,sog_out,identical_to_prv
126,7905584,Shahab 14,3792.0,27.108385,56.067524,2016-10-10 20:27:51,0.1,127,27.108412,56.067524,2016-10-11 09:41:22,0.1,NaN
902,7905584,Shahab 14,3792.0,27.107901,56.067810,2016-10-31 13:58:49,0.1,903,27.107988,56.067734,2016-11-01 02:28:46,0.1,False
1537,7905584,Shahab 14,3792.0,27.104183,56.069904,2016-11-28 15:28:00,0.0,1538,27.104074,56.069778,2016-11-29 06:29:52,0.8,False
7235,8209729,MSC Perle,61.0,22.737938,69.690994,2016-11-22 04:29:08,0.9,7236,22.738213,69.690681,2016-11-22 20:40:03,0.0,False
7237,8209729,MSC Perle,3761.0,25.023144,55.050079,2016-11-26 07:45:02,0.0,7238,25.022978,55.049988,2016-11-27 00:59:02,0.0,False
